# Text Analytics Simplified

In this demo we will showcase the powerful capabilities of the Data Ninja services by contructing a Text Analytics pipeline from scratch. By combining open source tools and packages with Data Ninja we will show you how the sematic content from unstructured data can be easily obtained and leveraged in your analytics pipeline.

We will walk through the following steps:

### 1. Fetch trending URLs

First we will scrape a news aggregation website (in this case Google News, but the idea can be extended to other news sites as well) and obtain a list of URLs that point to valid news articles.

### 2. Extract article text from URLs

We will show how the Data Ninja text extraction service can be used to identify and extract the main text from an HTML page removing all the boilerplate content (such as running headers/footers, menus, ads). 

### 3. Extract semantic content from article text

Once the text has been extracted from a webpage, we will then use another Data Ninja service to tag the article with entities and sentiment. Our content tagging system is capable of identifying the broader context of an article as we will show.

### 4. Clustering to find topics

Semantic content extracted from the text can be utilized as features and Machine Learning techniques can be used to derive isights from unstructured data. One simple example is to use a common text clustering technique like LDA to identify the topics from a collection of articles.

### 5. Visualization

The insights obtained from previous stages can be communicated using a graphical visualization library. We will show a new Data Ninja app called Newsbot Ninja that brings many of these ideas together: https://newsbot.dataninja.net/

Let's get started!

## Demo

We will scrape a news aggregator site (Google News) to collect a list URLs that point to trending news articles. We need to remove links that are not likely to be news articles from popular media sites (such as Wikipedia or Youtube).

#### Helper methods for harvesting the links

In [ ]:
from bs4 import BeautifulSoup
import requests

# Sites to exclude from our trending news URL collection
exclusions = ['google.com','youtube.com','wikipedia.org','blogspot.com']
prefix = 'http://'

def include_url(url):
    for excl in exclusions:
        if url.find(excl) > 0:
            return False
    return True

# Fetch the page content and extract the links
def fetch_links(url):
    response = requests.get(prefix + url)
    page_content = response.text
    soup = BeautifulSoup(page_content, "lxml")
    links = soup.find_all('a')
    return links

#### Collect and print the set of links

In [ ]:
linkset = set()
links = fetch_links('news.google.com')
    
# Collect the article links applying the URL filters
for link in links:
    href = link.get('href')
    if str(href).startswith(prefix) and include_url(str(href)):
        linkset.add(link.get('href').strip())
        print str(href)

print 'Links harvested: ', str(len(linkset))

# Take 100 links for the demo
links100 = list(linkset)[:100]

#### Accessing the Data Ninja services

Please sign-up at https://market.mashape.com/dataninja/smart-content and obtain your free Data Ninja API key. We will access the Smart Content service to analyze the semantic content of each article obtained in the previous step. The Smart Content service is based on our pre-built knowledge graph database.

Alternatively, you can use the Amazon Web Serivices API Gateway to access our services (using your AWS account): https://auth.dataninja.net/cart

In [ ]:
import json

with open('mashape_key.txt', 'r') as keyfile:
    mashape_key = keyfile.read().rstrip()
    
# Please add your own Data Ninja API Mashape key here -->
# mashape_key = <your-own-mashape-key>
        
smartcontent_url = 'https://smartcontent.dataninja.net/smartcontent/tag'
headers = {'Content-Type': 'application/json', 
           'Accept': 'application/json',
           'X-Mashape-User': 'Newsbot',
           'X-Mashape-Key': mashape_key}

# If you are using AWS API Gateway, please add the X-API-Key: <your-AWS-key> 
# in place of 'X-Mashape-Key': mashape_key and use the following link to access 
# the service: https://api.dataninja.net/smartcontent/tag

def fetch_smartcontent(link):
    payload = {'url': link, 'max_size': 10}
    response = requests.post(smartcontent_url, headers=headers, data=json.dumps(payload))
    return response.json()


#### Fetch Smart Content

The Smart Content service analyzes the text to produce concepts, categories, keywords and sentiments in JSON output format. Here is an example:

http://www.nba.com/warriors/news/warriors-announce-tv-radio-schedule-western-conference-finals

In [ ]:
data = fetch_smartcontent('http://www.nba.com/warriors/news/warriors-announce-tv-radio-schedule-western-conference-finals')

# Display the JSON output from Smart Content
print json.dumps(data, indent=4)

#### Article Text Extraction

Built into our Smart Content Service is the ability to extract the main text from a web page using machine learning techniques. Here is an example:

In [ ]:
# Dispay the extracted text from Smart Content
print data['text']

#### Fetch Smart Content for a set of links

Now we will extract the Smart Content for the list of URLs we obtained from Google News earlier. We will specifically prepare a list of extracted text for topic clustering in the next step.

In [ ]:
import json

# Call the Smart Content service and collect the article text into a list
documents = []

for link in links100:    
    data = fetch_smartcontent(link)
    if 'text' in data and len(data['text']) > 100:
        documents.append(data['text'])

print 'Documents in collection: ', str(len(documents))

#### Clustering text to find topics

The rich semantic content returned by Data Ninja services can be used for solving many Text Analytics problems and derive additional insights. Here will show you how topic clustering can be done with the extracted text using the standard LDA algorithm from the Gensim library. 

In [ ]:
from gensim import corpora, models
import re

# Prepare a set of stopwords and other tokens to be removed from stream
stoplist = set([
    'a', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', 'came', 'can', 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly ', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', 'course','currently', 'd', 'definitely', 'described', 'despite', 'did', 'different', 'do', 'does', 'doing', 'done', 'down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight', 'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'f', 'far', 'few', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'former', 'formerly', 'forth', 'four', 'from', 'further', 'furthermore', 'g', 'get', 'gets', 'getting', 'given', 'gives', 'go  ', 'goes', 'going', 'gone', 'got', 'gotten', 'greetings', 'h', 'had', 'happens', 'hardly', 'has', 'have', 'having', 'he', 'hello', 'help', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'i', 'ie', 'if','ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'insofar', 'instead', 'into', 'inward', 'is', 'it', 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', 'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks', 'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean', 'meanwhile', 'merely', 'might', 'more', 'moreover', 'most', 'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'non', 'none', 'noone', 'nor', 'normally', 'not', 'nothing', 'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old','on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'p', 'particular', 'particularly', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provides', 'q', 'que', 'quite', 'qv', 'r', 'rather', 'rd', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', 'she', 'should', 'since', 'six', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 't','take', 'taken', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'theres', 'thereupon', 'these', 'they', 'think', 'third', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless', 'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants', 'was', 'way', 'we', 'welcome', 'well', 'went', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby','wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', 'wonder', 'would', 'would', 'x', 'y', 'yes', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', '', 'am', 'pm', 'mr', 'hd', 'vr', 'top', 'new','z', 'zero', '-', '--', '|'
])

stoplist |= set(map(str, list(range(0, 3000))))

#### Run LDA on our news article collection

We have developed our own community detection and topic clustering pipeline that takes full advantage of the rich sematic content provided by the Data Ninja services.

You can take a look at our Newsbot Ninja App here: https://newsbot.dataninja.net/

In [ ]:
def get_topics(num_topics, documents):
    texts = [[word for word in re.split(r'\W+', document.lower()) 
              if word.strip() not in stoplist]
             for document in documents]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    # Run the LDA model 
    lda = models.ldamodel.LdaModel(corpus=corpus, 
                                   id2word=dictionary, 
                                   num_topics=num_topics, 
                                   update_every=1, 
                                   passes=10)
    
    # Collect the LDA output for display
    result = {}
    for i in xrange(num_topics):
        result[i] = lda.show_topic(i, 5) # Get only the top five labels per topic
    return result        

In [ ]:
# Run LDA for our news article collection and display the results
topics = get_topics(num_topics=20, documents=documents)

print "{:<10} {:<15} {:<10}".format('Topic Id','Label','Probability')
print '-' * 45

for k, v in topics.iteritems():
    for item in v:
        print "{:<10} {:<15} {:<10}".format(k, item[0], item[1])
    print '-' * 45